In [ ]:
%reload_ext autoreload
%autoreload 2


import functools
print = functools.partial(print, flush=True)

import os
import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
import matplotlib.pyplot as plt
from matplotlib import cm
from pathlib import Path
import pickle
from tqdm import tqdm
import scipy

import flexiznam as flz
from cottage_analysis.preprocessing import synchronisation
from cottage_analysis.analysis import spheres, find_depth_neurons, common_utils, fit_gaussian_blob, size_control
from cottage_analysis.plotting import basic_vis_plots, plotting_utils
from cottage_analysis.pipelines import pipeline_utils
from v1_depth_analysis.v1_manuscript_2023 import depth_selectivity, rf, get_session_list

In [ ]:
VERSION = 2
SAVE_ROOT = '/camp/lab/znamenskiyp/home/shared/presentations/v1_manuscript_2023/ver'+str(VERSION)+'/fig2/'
os.makedirs(SAVE_ROOT, exist_ok=True)

In [ ]:
# Load data
# Load data for example reconstructed stimulus frame
project='hey2_3d-vision_foodres_20220101'
flexilims_session = flz.get_flexilims_session(project)

session_name_example_frame='PZAH6.4b_S20220419'
frames_example_frame, _ = spheres.regenerate_frames_all_recordings(
    session_name=session_name_example_frame,
    flexilims_session=flexilims_session,
    project=None,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=2,
    return_volumes=True,
    resolution=1,
)

# Load data for example RF
session_name_rf='PZAH8.2h_S20230116'
vs_df_rf, trials_df_rf = spheres.sync_all_recordings(
    session_name=session_name_rf,
    flexilims_session=flexilims_session,
    project=None,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=5,
    return_volumes=True,
)
frames_rf, imaging_df_rf = spheres.regenerate_frames_all_recordings(
    session_name=session_name_rf,
    flexilims_session=flexilims_session,
    project=None,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=5,
    return_volumes=True,
    resolution=5,
)
neurons_ds_rf = pipeline_utils.create_neurons_ds(
    session_name=session_name_rf,
    flexilims_session=flexilims_session,
    project=project,
    conflicts="skip",
)
neurons_df_rf = pd.read_pickle(neurons_ds_rf.path_full)



In [ ]:
# # Load data for significant RF %
project='hey2_3d-vision_foodres_20220101'
flexilims_session = flz.get_flexilims_session(project)
all_sig, all_sig_ipsi = rf.load_sig_rf(flexilims_session=flexilims_session, 
                                       session_list= 
                                       get_session_list.get_all_sessions(project=project, mouse_list=["PZAH6.4b", 
                                                                                                    #   "PZAG3.4f",
                                                                                                      "PZAH8.2h", 
                                                                                                      "PZAH8.2h", 
                                                                                                      "PZAH8.2i",
                                                                                                      "PZAH10.2d",
                                                                                                      "PZAH10.2f"
                                                                                                      ], closedloop_only=False, openloop_only=False),
                                       n_std=6)

In [ ]:
fig = plt.figure(figsize=(7.08,7.08))
fontsize_dict = {'title': 7, 'label': 7, 'tick': 5, 'legend':5}
# Fig.2A Example reconstructed stimuli frame
rf.plot_stimuli_frame(fig=fig, 
                   frames=frames_example_frame[:,:,frames_example_frame.shape[2]//2:], 
                   iframe=10000, 
                   idepth=3, 
                   ndepths=8, 
                   plot_x=0,
                   plot_y=0.8,
                   plot_width=0.3,
                   plot_height=0.4,
                   plot_prop=0.9, 
                   fontsize_dict=fontsize_dict)   

        
# Fig.2B Linear-nonlinear receptive field model schematic. (maybe not necessary)

# Fig.2C Example receptive field of 2 V1 neurons
# PZAH8.2h_S20230116: 271, 299, 319, 321, 402, 344, 356, 306, 110, 117, 86, 88, 93, 110, 117, 120, 141, 223, 250, 350, 370, 412, 434, 437
SELECT_ROIS = [591, 344, 88] #change 271 to 591
for iroi, roi in enumerate(SELECT_ROIS):
    depth_selectivity.plot_depth_tuning_curve(
        fig=fig,
        neurons_df=neurons_df_rf,
        trials_df=trials_df_rf,
        roi=roi,
        rs_thr=0.2,
        plot_fit=False,
        linewidth=2,
        linecolor="k",
        fit_linecolor="r",
        closed_loop=1,
        plot_x=0.32+0.15*iroi,
        plot_y=1,
        plot_width=0.05,
        plot_height=0.05,
        fontsize_dict=fontsize_dict,
        )
    plt.ylabel('')
    plt.xlabel('')
    plt.gca().set_xticklabels([])
    plt.gca().set_yticklabels([])

    if iroi == len(SELECT_ROIS)//2: 
        xlabel='Azimuth (deg)'
    else:
        xlabel=''
    if iroi == 0:
        ylabel='Elevation (deg)'
    else:
        ylabel=''
    rf.plot_rf(
        fig=fig,
        neurons_df=neurons_df_rf,
        roi=roi, 
        ndepths=8,
        frame_shape=(16,24),
        plot_x=0.1+0.15*iroi,
        plot_y=0.9,
        plot_width=0.5,
        plot_height=0.5,
        plot_prop=0.9,
        xlabel=xlabel,
        ylabel=ylabel,
        fontsize_dict=fontsize_dict,
    )

# Fig.2D Proportion of neurons with significant receptive fields across imaging sessions
rf.plot_sig_rf_perc(fig=fig,
               all_sig=all_sig,
               all_sig_ipsi=all_sig_ipsi,
               plot_x=0.8,
               plot_y=0.45,
               plot_width=0.2,
               plot_height=0.5,
               bar_color='k',
               scatter_color='k',
               scatter_size=10,
               scatter_alpha=0.3,
               fontsize_dict=fontsize_dict,
               )

# Fig.2F-G Preferred azimuth & elevation follow V1 retinotopic gradients
# example session, spatial distribution of preferred azimuth/elevation/depth 
# visualize azimuth & elevation on the 2p cell mask (or as a function of position)--> color-code - two separate plots --> include position in neurons_df
_,_ = depth_selectivity.plot_example_fov(
    fig=fig,
    flexilims_session=flexilims_session,
    session=session_name_rf,
    neurons_df=neurons_df_rf,
    ndepths=8,
    param="preferred_depth",
    cmap=cm.cool.reversed(),
    background_color = np.array([0.133,0.545,0.133]),
    plot_x=0.3,
    plot_y=0.15,
    plot_width=0.2,
    plot_height=0.2,   
    fontsize_dict=fontsize_dict,
)
plt.title('Preferred depth', fontsize=fontsize_dict["title"])

azi, ele = depth_selectivity.plot_example_fov(
    fig=fig,
    flexilims_session=flexilims_session,
    session=session_name_rf,
    neurons_df=neurons_df_rf,
    ndepths=8,
    param="preferred_azimuth",
    cmap=cm.summer,
    background_color = np.array([0.133,0.545,0.133]),
    plot_x=0.55,
    plot_y=0.15,
    plot_width=0.2,
    plot_height=0.2,       
    fontsize_dict=fontsize_dict,
)
plt.title('Preferred azimuth', fontsize=fontsize_dict["title"])

_, _ = depth_selectivity.plot_example_fov(
    fig=fig,
    flexilims_session=flexilims_session,
    session=session_name_rf,
    neurons_df=neurons_df_rf,
    ndepths=8,
    param="preferred_elevation",
    cmap=cm.summer,
    background_color = np.array([0.133,0.545,0.133]),
    plot_x=0.8,
    plot_y=0.15,
    plot_width=0.2,
    plot_height=0.2,   
    fontsize_dict=fontsize_dict,
)
plt.title('Preferred elevation', fontsize=fontsize_dict["title"])
plt.savefig(SAVE_ROOT+'fig2.pdf', transparent=True, bbox_inches='tight')
plt.savefig(SAVE_ROOT+'fig2.png', transparent=True, bbox_inches='tight', dpi=300)



In [ ]:
# MINOR:
# - add location on overview & scalebars (can highlight the neurons above)
# - DONE: change the near cell example to 591
# - DONE: change colormap for azimuth/elevation to plasma, viridis etc summer autumn, BuYl

# CHECK:
# - double check if significant RF cells include optic flow near cells

# MAJOR:
# - summary of all sessions: compute the distance between n nearest neighbors vs n random neighbors. sklearn.neighbors.KDTree

# Other:
# - R2 distribution, add non-linear fit (no need to do it yet);

### SUPP

In [ ]:
# Fig.2E Distribution of azimuth/elevation of depth-selective neurons in 2 example imaging sessions
# Load data for plotting center of RFs
sessions_dist_exp = ['PZAH6.4b_S20220419','PZAH6.4b_S20220426']
results_dist_exp = rf.get_rf_results(project, sessions_dist_exp)

fig=plt.figure()
rf.plot_rf_centers(fig=fig,
                    results=results_dist_exp, 
                    colors=['r','b'],
                    ndepths=5,
                    frame_shape=(16,24),
                    n_stds=5,
                    plot_x=0.3,
                    plot_y=0,
                    plot_width=0.3,
                    plot_height=0.3,
                    fontsize_dict={'title': 15, 'label': 10, 'tick': 5},
                    )


In [ ]:
plt.figure()
plt.hist(azi, bins=20)
plt.gca().set_aspect(0.6)
plotting_utils.despine()
plt.xlabel("Preferred azimuth (°)")

plt.figure()
plt.hist(ele, bins=20)
plt.gca().set_aspect(0.3)
plotting_utils.despine()
plt.xlabel("Preferred elevation (°)")